<a href="https://colab.research.google.com/github/HuangJin-De/am_NH/blob/main/discussion/20231115.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! rm -rf am_NH
! git clone https://github.com/HuangJin-De/am_NH.git

Cloning into 'am_NH'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 164 (delta 64), reused 110 (delta 35), pack-reused 0
Receiving objects: 100% (164/164), 6.03 MiB | 6.55 MiB/s, done.
Resolving deltas: 100% (64/64), done.


In [2]:
import numpy as np
import matplotlib.pyplot as plt

path='am_NH'
filen=path+'/train_data/zm_10days_data.dat'

data=np.fromfile(filen,dtype=np.float32)
data=data.reshape(-1,42)

y1=np.copy(data[:,0])
print(y1[0:10])
y2=np.copy(data[:,1])
print(y2[0:10])
x=np.copy(data[:,2:43])

print(y1.shape,y2.shape,x.shape)


[ 0.0833049  -1.0930462  -1.3138411  -0.75141543 -0.5470629   0.33093917
  0.02753482 -1.3000826   0.44190055  1.824813  ]
[ 0.81427157 -0.04958611 -0.70947564  0.5993808   1.0594871  -0.33536747
 -1.3672273  -0.12659782 -0.5226433  -0.6045729 ]
(4662,) (4662,) (4662, 40)
